In [1]:
import torch

In [2]:
x = torch.ones(2, 2, requires_grad=True)
print(x)
print(x.grad_fn)
y = x + 2
print(y)
print(y.grad_fn)
z = y * y * 3
out = z.mean()
print(z)
print(out)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
None
tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)
tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
tensor(27., grad_fn=<MeanBackward0>)


In [3]:
"""
因为以上out是一个标量，所以调用backward()时不需要指定求导变量
"""
out.backward()  # 等价于out.backward(torch.tensor(1.0))
"""
out是一个关于z的梯度为全1/4矩阵？
out.backward(torch.tensor(1.)) 计算out关于所有具有requires_grad=True的张量的梯度。在这个例子中，只有x
具有requires_grad=True.
out是一个标量，它是通过对z取平均值得到的，因此，out关于z的梯度是一个全1/4矩阵，即每个元素对out的贡献都是
相等的
"""

'\nout是一个关于z的梯度为全1/4矩阵？\nout.backward(torch.tensor(1.)) 计算out关于所有具有requires_grad=True的张量的梯度。在这个例子中，只有x\n具有requires_grad=True.\nout是一个标量，它是通过对z取平均值得到的，因此，out关于z的梯度是一个全1/4矩阵，即每个元素对out的贡献都是\n相等的\n'

In [4]:
"""
z = y*y*3 = (x+2)(x+2)*3
z关于y的梯度是6y
y关于x的梯度为全 1 矩阵？
y是通过y=x+2计算得到的，这意味着y中的每个元素都是x中对应元素加上2得到的。因此，y中的每个元素都只依赖于x中的
对应元素，而与其他元素无关。
根据导数的定义，y关于x的梯度表示y中每个元素关于x中对应元素的偏导数。由于y=x+2，所以y中每个元素关于x中对应元素
的偏导数都为1，因此，y关于x的梯度为全1的矩阵

""" 
print(x.grad)

"""
x.grad=6y/4=6(x+2)/4=4.5, 即一个2×2的全4.5的矩阵
"""

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


'\nx.grad=6y/4=6(x+2)/4=4.5, 即一个2×2的全4.5的矩阵\n'

In [ ]:
"""
什么是反向传播？
反向传播是一种高效计算神经网络中所有权重的梯度的方法，它通过输出层开始，逐层向前计算每层的梯度，来实现对
所有权重的梯度的计算。这种方法利用了链式法则，将每一层的梯度表示为下一层梯度的函数，从而避免了对每个权重
单独计算梯度所需的大量计算。

在线性回归模型中，反向传播就是根据给定的损失函数，计算模型参数的梯度，并使用梯度下降算法来更新模型参数的过程。
"""

In [ ]:
"""def grad(w, b, X, Y):
    # np.mean((Y - (w * X + b)) * X)就相当于1/n * Σ(yi-(wx+b)) * x
    dw = -2 * np.mean((Y - (w * X + b)) * X)
    db = -2 * np.mean(Y - (w * X + b))
    return dw, db"""

"""
上面grad函数是一个手动计算线性回归模型中权重w和偏置b的梯度的函数，它通过显式计算损失函数关于w和b的偏导数
来得到梯度。
反向传播是一种自动计算神经网络中所有权重的梯度的方法，它不需要显式第写出损失函数关于每个权重的偏导数，而是
通过从输出层开始，逐层向前计算每一层的梯度，来实现对所有权重的梯度的计算。
相比于手动计算梯度，反向传播的优点在于它能够自动化第处理复杂的神经网络结构，能够高效地计算所有权重的梯度。
这样，我们就可以避免手动推导复杂的偏导数公式，并且能够快速地更新神经网络中的所有权重
"""

In [10]:
# 反向传播示例代码，实现了一个简单的线性回归模型

import torch

# 定义输入和输出数据
x = torch.tensor([[1.0], [2.0], [3.0]])
y = torch.tensor([[2.0], [4.0], [6.0]])

# 定义模型
model = torch.nn.Linear(1, 1)

# 定义损失函数
loss_fn = torch.nn.MSELoss()

# 定义优化器
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# 训练模型
for t in range(1000):
    # 前向传播
    y_pred = model(x)

    # 计算损失
    loss = loss_fn(y_pred, y)

    # 反向传播
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# 测试模型
x_test = torch.tensor([[4.0]])
y_test = model(x_test)
print(y_test)

"""
这段代码定义了一个简单的线性回归模型，使用均方误差作为损失函数，并使用随机梯度下降算法进行优化。
在每次迭代中，代码先进行前向传播计算预测值，然后计算损失，最后通过调用loss.backward()进行反向
传播计算梯度，并使用优化器更新模型参数
"""
"""
前向传播是指根据给定的输入数据和模型参数，计算模型的预测输出的过程。
在线性回归模型中，前向传播指的是根据给定的输入数据和模型参数，计算模型的预测输出的过程。

线性回归模型的形式通常为 y = wx + b，其中 x 是输入数据，w 是权重，b 是偏置。在前向传播过程中，我们将输入
数据 x 乘以权重 w，然后加上偏置 b，得到模型的预测输出 y。
举个例子，假设我们有一个一元线性回归模型，权重 w 为 2，偏置 b 为 1。现在，假设我们有一个输入数据 x = 3。
在前向传播过程中，我们将输入数据乘以权重，然后加上偏置，得到预测输出 y = 2 * 3 + 1 = 7。
"""

tensor([[7.9848]], grad_fn=<AddmmBackward0>)


In [ ]:
"""
雅克比矩阵(Jacobian matrix)是一个向量值函数的所有一阶偏导数组成的矩阵，它可以是一个矩形矩阵，也可以是一个方阵，

"""

In [ ]:
"""
导数表示函数在某一点处的瞬时变化率，即函数在该点处的切线斜率。
微分表示函数在某一点处(趋近于无穷小)的变化量，是一种变化的量。
例如，假设我们有一个函数 f(x)，它在 x=a 处的导数为 f′(a)。那么，当 x 从 a 变化到 a+Δx 时，f(x) 的变化量 Δy 可以近似表示为：
Δy≈f′(a)Δx

其中，Δx表示x的变化量，Δy表示f(x)的变化量，而 f′(a)Δx 就是 f(x) 在 x=a 处的微分， 通常记作dy,因此，
微分可以看作是函数在某一点处的线性逼近。
"""

"""
自动微分是一种借由计算机程序计算一个函数导数的方法，
自动微分使用这个事实：任何实作一个向量函数y=F(x)的计算机程序，一般而言，可以被分解成由指定运算所成的序列，
而其中每一个都可以借由查表而轻易地微分，这些计算某一特定项的‘基本偏微分’是依照微积分中的符合函数求导法则
来合成某个F的微分资讯(如梯度，切线，雅克比矩阵等)，这个过程会产生确实（数值上准确）的导数，因为只在最基础
的层面做符号转换，自动微分避免了复杂的符号运算的问题

"""